# Chains

**Chains** in LangChain allow you to combine multiple components (such as prompts, LLMs, tools, and output parsers) into a single workflow.  
They help structure complex LLM applications by breaking tasks into clear, manageable steps.


In [7]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature=0.7,
    max_new_tokens=700,
)

model = ChatHuggingFace(llm=llm)

llm2 = HuggingFaceEndpoint(
    repo_id="MiniMaxAI/MiniMax-M2.1",
    task="text-generation",
    max_new_tokens=1000,
)

model2 = ChatHuggingFace(llm=llm2)

## Simple Chains

**Simple Chains** execute steps sequentially, where the output of one step becomes the input to the next step.

### Why use Simple Chains?

- They are easy to understand and implement.
- They are useful for linear workflows.
- They are ideal for basic tasks like summarization or question answering.

**Example use cases:**  
Text summarization, translation, sentiment analysis.


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template='Generate five fact about {topic} in numerical points',
    input_variables=['topic']
)

parser = StrOutputParser()

chain = prompt | model | parser

response = chain.invoke({'topic':'Cricket'})

print(response)

 1. Cricket is believed to have originated in South Asia around the 13th or 14th century. Here are some fascinating numbers related to this sport:

   a. More than 2.5 billion people around the world follow cricket.
   
   b. Cricket is the most popular sport in countries like India, Australia, Pakistan, and New Zealand.
   
   c. There are 108 cricket playing countries recognized by the International Cricket Council (ICC).
   
   d. The first recorded cricket match was between two parishes, Exeter and Bradfield, in Devon, England in 1598.

2. Cricket is a bat-and-ball game played between two teams of eleven players each. Here are some important numbers in cricket:

   a. Each team has eleven players, including one wicket-keeper.
   
   b. The objective of the batting team is to score as many runs as possible, while the fielding team tries to prevent them from doing so.
   
   c. A cricket match typically lasts for three to five days, although Twenty20 matches can be completed in a sin

In [4]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
   +-----------------+     
   | ChatHuggingFace |     
   +-----------------+     
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


## Sequential Chains

**Sequential Chains** are a type of chain where multiple steps are executed in a fixed order, and each step’s output is passed as input to the next step.

### Why use Sequential Chains?

- They help break complex tasks into smaller, ordered steps.
- They make multi-step reasoning easier to manage and debug.
- They are useful when later steps depend on earlier outputs.

**Example use cases:**  
Generating an outline first and then expanding it, summarizing content before analysis, or refining LLM-generated responses step by step.

In [5]:
prompt1 = PromptTemplate(
    template='Generate detail one paragraph about {topic} around in 377 words.',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate Seven Line summary and each line around 17 words from the text \n {text}',
    input_variables=['text']
)

chain = prompt1 | model | parser | prompt2 | model | parser

res = chain.invoke({'topic': 'Unemployement In India'})

print(res)

 1. Unemployment in India is a long-standing socio-economic challenge with a workforce of approximately 500 million in 2021.
2. Unemployment rate was 6.5% in 2019, amounting to about 193 million unemployed persons, urban and rural, formally and informally.
3. Skills mismatch between labor force and market demands, with agriculture employing a large portion despite low productivity and seasonal work.
4. Education system criticized for inadequate skills training, particularly in sectors like manufacturing, IT, and healthcare.
5. Informal sector accounts for large employment but lacks job security, social protection, and benefits, leaving many without safety nets.
6. Regional disparities exist, with lower unemployment in industrialized states versus higher rates in less developed areas.
7. Government initiatives like PMEGP and NSDM aim to generate self-employment, skill workforce, and invest in infrastructure, but challenges persist due to skills gap, labor market issues, and regional dis

In [6]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
   +-----------------+     
   | ChatHuggingFace |     
   +-----------------+     
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

## Parallel Chains

**Parallel Chains** execute multiple chains at the same time using the same input.

### Why use Parallel Chains?

- They improve efficiency by running independent tasks concurrently.
- They allow extraction of multiple outputs from a single input.
- They are useful when tasks do not depend on each other.

**Example use cases:**  
Extracting sentiment, keywords, and summary simultaneously.


In [8]:
text = """
Most modern deep learning models are based on multi-layered neural networks such as convolutional neural networks and transformers, although they can also include propositional formulas or latent variables organized layer-wise in deep generative models such as the nodes in deep belief networks and deep Boltzmann machines.

Fundamentally, deep learning refers to a class of machine learning algorithms in which a hierarchy of layers is used to transform input data into a progressively more abstract and composite representation. For example, in an image recognition model, the raw input may be an image (represented as a tensor of pixels). The first representational layer may attempt to identify basic shapes such as lines and circles, the second layer may compose and encode arrangements of edges, the third layer may encode a nose and eyes, and the fourth layer may recognize that the image contains a face.

Importantly, a deep learning process can learn which features to optimally place at which level on its own. Prior to deep learning, machine learning techniques often involved hand-crafted feature engineering to transform the data into a more suitable representation for a classification algorithm to operate on. In the deep learning approach, features are not hand-crafted and the model discovers useful feature representations from the data automatically. This does not eliminate the need for hand-tuning; for example, varying numbers of layers and layer sizes can provide different degrees of abstraction.
"""

In [10]:
from langchain_classic.schema.runnable import RunnableParallel

prompt1 = PromptTemplate(
    template='Generate Short and simple two paragraph summary from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 7 short question and answer from the followin summary \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into single document \n notes:{notes} and quiz:{quiz}',
    input_variables=['notes', 'quiz']
)

parallel_chain = RunnableParallel({
        'notes': prompt1 | model | parser,
        'quiz': prompt2 | model2 | parser
    })

merge_chain = prompt3 | model | parser

chain = parallel_chain | merge_chain

document = chain.invoke({'text': text})
print(document)

 # Deep Learning Notes and Q&A

 Deep learning is a type of machine learning that uses a hierarchy of layers to transform input data into more abstract and composite representations. Modern deep learning models are primarily based on multi-layered neural networks, including convolutional neural networks (CNNs), transformers, and deep generative models like deep belief networks and deep Boltzmann machines.

**Q1: What are the foundational structures of modern deep learning models?**
A: Modern deep learning models use neural networks as their primary structure. These models can be further divided into convolutional neural networks (CNNs), transformers, and deep generative models like deep belief networks and deep Boltzmann machines.

**Q2: How does deep learning transform input data through its architecture?**
A: Deep learning uses a hierarchy of layers to transform input data into progressively more abstract and composite representations. Each successive layer builds upon the previous o

In [11]:
chain.get_graph().print_ascii()

              +---------------------------+              
              | Parallel<notes,quiz>Input |              
              +---------------------------+              
                  ***               ***                  
               ***                     ***               
             **                           **             
+----------------+                    +----------------+ 
| PromptTemplate |                    | PromptTemplate | 
+----------------+                    +----------------+ 
          *                                   *          
          *                                   *          
          *                                   *          
+-----------------+                  +-----------------+ 
| ChatHuggingFace |                  | ChatHuggingFace | 
+-----------------+                  +-----------------+ 
          *                                   *          
          *                                   *          
          *   

## Conditional Chains

**Conditional Chains** execute different chains based on a condition or decision.

### Why use Conditional Chains?

- They enable branching logic in workflows.
- They allow different processing paths for different inputs.
- They are useful for dynamic, decision-based applications.

**Example use cases:**  
Routing queries to different models, handling user intent, or applying different logic based on sentiment.

In [20]:
from langchain_classic.schema.runnable import RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing_extensions import Literal

with open('reviews.txt', 'r') as file:
    reviews = [review.strip() for review in file.readlines()][:2]

class FeedBack(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

structure_output = PydanticOutputParser(pydantic_object=FeedBack)

prompt = PromptTemplate(
    template='Classify the sentiment of the following feedback text \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction': structure_output.get_format_instructions()}
)

prompt1 = PromptTemplate(
    template='Write an appropriate response to this positive feedback in about 29 words \n {feedback}',
    input_variables=['feedback'],
)

prompt2 = PromptTemplate(
    template='Write an appropriate response to this negative feedback in about 29 words  \n {feedback}',
    input_variables=['feedback'],
)

sentiment_chain = prompt | model2 | structure_output

branch_chain = RunnableBranch(
    (lambda x:x.sentiment=='positive', prompt1 | model2 | parser),
    (lambda x:x.sentiment=='negative', prompt2 | model2 | parser),
    RunnableLambda(lambda x: 'Could not find sentiment')
)

chain = sentiment_chain | branch_chain

for review in reviews:
    res = chain.invoke({'feedback':review})
    print(res)

Thank you so much for your wonderful feedback! We're thrilled you enjoyed it and appreciate your kind words. Please let us know if there's anything else we can do!
We apologize for this unsatisfactory experience. Your feedback helps us improve, and we're fully dedicated to resolving this issue. Please reach out so we can make it right.


In [21]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
  +-----------------+    
  | ChatHuggingFace |    
  +-----------------+    
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
